Tester et entraîner un modèle HTR
===


# Tester les performances du modèle

Les recueils qui constituent une part importante de la correspondance de Constance de Salm sont l'oeuvre de plusieurs scribes. La différence des écritures représente une difficulté certaine pour la reconnaissance automatique.

## Classer les images par mains

**compléter**

Renseigner le fichier [mains.csv](./sources/mains.csv) qui jouera un rôle important dans l'évaluation des modèles et leur entraînement.

## Sélectionner des spécimens de chaque écriture

On crée dans le dossier de chaque main un dossier **test** contenant des spécimens d'écriture selon les critères suivants :
- Reproductions de bonne qualité (sans problème de transparence)
- 2 doubles pages environ
- Choisies manière discontinue (l'écriture d'une même main peut en effet varier)

Si certaines mains ne sont attestées qu'en compagnie d'autres écritures, on veillera à limiter le test de reconnaissance d'écriture aux seuls lignes de la main à tester (en supprimant après segmentation les lignes non pertinentes afin que la reconnaissance de l'écriture ne les traite pas).


## Tester des modèles HTR

### Installer l'application Kraken

### Importer un modèle

### Initier un journal de tests

Le script [journalReconn.py](./py/journalReconn.py) permet de pré-remplir un journal pour l'enregistrement des résultats des tests effectués sur les modèles. 

On lui donne comme argument un nom de modèle et il écrit dans le fichier Json [journal-reconn.json](./sources/journal-reconn.json), à la date et à l'heure courante, pour chaque main listée dans le fichier [mains.csv](./sources/mains.csv).

Le test initial se fait naturellement sans avoir entraîné le modèle sur les vérités de terrain.

Pour lancer l'écriture du journal :

In [9]:
!python3 py/journalReconn.py cm_ft_mrs15_11.mlmodel

Le contenu écrit dans le fichier journal donne comme **accuracy** pour chaque main une valeur de 0. Cette valeur doit être saisie manuellement dans le fichier une fois effectué le test comme suit.


# Effectuer un test

Avec la commande suivante on effectue un test d'acuité pour un modèle sur une main particulière (on doit relever l'*Average accuracy*) :

In [ ]:
# ketos test -m ./source/modeles/NOM-MODEL.mlmodel source/NOM-MAIN/test/*xml -f alto
!ketos test -m ./sources/modeles/souvay.mlmodel sources/mainCdS02_Konv019_02/test/*xml -f alto

Loading model ./sources/modeles/souvay.mlmodel	✓
Evaluating ./sources/modeles/souvay.mlmodel
Evaluating ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸━━━━━━━━━  77% 68/88 0:00:05 0:00:17

## Créer des vérités de terrain pour chaque spécimen

On procède à la constitution d'une vérité de terrain pour chaque spécimen. Pour cela, il n'est pas nécessaire d'annoter les régions d'écriture dans le cadre de la segmentation de la page (comme expliqué dans le notebook [suivant](./2_Realiser_une_prediction_HTR.ipynb)). On se contente d'une segmentation automatique, puis on transcrit les lignes manuellement.